# BuildingPit Element

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import timml as tml

## Parameters
Define some parameters 

In [ ]:
kh = 2.0  # m/day
f_ani = 1 / 10  # anisotropy factor
kv = f_ani * kh
ctop = 800.0  # resistance top leaky layer in days

In [ ]:
ztop = 0.0  # surface elevation
z_well = -13.0  # end depth of the wellscreen
z_dw = -15.0  # bottom elevation of sheetpile wall
z_extra = z_dw - 15.0  # extra layer
zbot = -60.0  # bottom elevation of the model

In [ ]:
length = 40.0  # length building pit in m
width = 30.0  # width building pit in m

h_bem = -6.21  # m
offset = 5.0  # distance groundwater extraction element from sheetpiles in m

In [ ]:
xy = [
    (-length / 2, -width / 2),
    (length / 2, -width / 2),
    (length / 2, width / 2),
    (-length / 2, width / 2),
    (-length / 2, -width / 2),
]

In [ ]:
for x, y in xy:
    (p2,) = plt.plot(x, y, "o", label="building pit pts")
plt.axis("equal")
plt.show()

## Model
Set up a model

In [ ]:
z = np.array([ztop + 1, ztop, z_dw, z_dw, z_extra, z_extra, zbot])
dz = z[1::2] - z[2::2]
dz

In [ ]:
kh_arr = kh * np.ones(dz.shape)

In [ ]:
c = np.r_[np.array([ctop]), dz[:-1] / (2 * kv) + dz[1:] / (2 * kv)]
c

Build model, solve, and calculate total discharge and distance to the 5 cm drawdown contour.

In [ ]:
ml = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)

layers = np.arange(np.sum(z_dw <= ml.aq.zaqbot))
last_lay_dw = layers[-1]

inhom = tml.BuildingPitMaq(
    ml,
    xy,
    kaq=kh_arr,
    z=z[1:],
    topboundary="conf",
    c=c[1:],
    order=4,
    ndeg=3,
    layers=layers,
)

tml.HeadLineSink(
    ml,
    x1=-length / 2 + offset,
    y1=width / 2 - offset,
    x2=length / 2 - offset,
    y2=width / 2 - offset,
    hls=h_bem,
    layers=np.arange(last_lay_dw + 1),
)
tml.HeadLineSink(
    ml,
    x1=-length / 2 + offset,
    y1=0,
    x2=length / 2 - offset,
    y2=0,
    hls=h_bem,
    layers=np.arange(last_lay_dw + 1),
)
tml.HeadLineSink(
    ml,
    x1=-length / 2 + offset,
    y1=-width / 2 + offset,
    x2=length / 2 - offset,
    y2=-width / 2 + offset,
    hls=h_bem,
    layers=np.arange(last_lay_dw + 1),
)

# ml.solve_mp(nproc=2)
ml.solve()

Qtot = 0.0

for e in ml.elementlist:
    if e.name == "HeadLineSink":
        Qtot += e.discharge()

print("\nDischarge =", np.round(Qtot.sum(), 2), "m^3/dag")

y = np.linspace(-width / 2 - 25, width / 2 + 1100, 201)
hl = ml.headalongline(np.zeros(201), y, layers=[0])
y_5cm = np.interp(
    -0.05, ml.headalongline(np.zeros(201), y, layers=0).squeeze(), y, right=np.nan
)
print("Distance to 5 cm drawdown contour =", np.round(y_5cm, 2), "m")

Plot an overview of the model

In [ ]:
ml.plots.topview()

Visualizations


In [ ]:
x = np.linspace(0.0, length / 2 + 1100, 201)
hl = ml.headalongline(x, np.zeros(201), layers=[last_lay_dw, last_lay_dw + 1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))

ax.plot(x, hl[0].squeeze(), label="head layer {}".format(last_lay_dw))
ax.plot(x, hl[1].squeeze(), label="head layer {}".format(last_lay_dw + 1))
ax.axhline(-0.05, color="r", linestyle="dashed", lw=0.75, label="-0.05 m")
ax.axhline(-0.5, color="k", linestyle="dashed", lw=0.75, label="-0.5 m")
ax.set_xlabel("x (m)")
ax.set_ylabel("head (m)")
ax.legend(loc="best")
ax.grid(True)

In [ ]:
x = np.linspace(-length / 2 - 25, 0.0, 201)
hl = ml.headalongline(x, np.zeros(201), layers=[last_lay_dw, last_lay_dw + 1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))

ax.plot(x, hl[0].squeeze(), label="head layer {}".format(last_lay_dw))
ax.plot(x, hl[1].squeeze(), label="head layer {}".format(last_lay_dw + 1))
ax.axhline(-0.05, color="r", linestyle="dashed", lw=0.75, label="-0.05 m")
ax.axhline(-0.5, color="k", linestyle="dashed", lw=0.75, label="-0.5 m")
ax.set_xlabel("x (m)")
ax.set_ylabel("head (m)")
ax.legend(loc="best")
ax.grid(True)

In [ ]:
xoffset = 50
zoffset = 15
x1, x2, y1, y2 = [-length / 2 - xoffset, 0.0, 0, 0]
nudge = 1e-6
n = 301

In [ ]:
# plot head contour cross-sections
h = ml.headalongline(
    np.linspace(x1 + nudge, x2 - nudge, n), np.linspace(y1 + nudge, y2 - nudge, n)
)
L = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
xg = np.linspace(0, L, n) + x1

zg = 0.5 * (ml.aq.zaqbot + ml.aq.zaqtop)
zg = np.hstack((ml.aq.zaqtop[0], zg, ml.aq.zaqbot[-1]))
h = np.vstack((h[0], h, h[-1]))

In [ ]:
levels = np.linspace(h_bem - 0.1, -0.0, 51)

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.set_aspect("equal")
ml.plots.topview(win=[x1, x2, y1, y2], orientation="ver", newfig=False)
cf = ax.contourf(xg, zg, h, levels)
cs = ax.contour(xg, zg, h, levels, colors="k", linewidths=0.5)
ax.set_ylim(z_dw - zoffset, z_dw + zoffset)
ax.set_ylabel("depth (m NAP)")
ax.set_xlabel("m")

cb = plt.colorbar(cf, ax=ax, shrink=0.6)
cb.set_label("head (m)")
cb.set_ticks(np.arange(-6, 0.1, 1))

## Model 2: Add more layers
Add more layers to the model to get a more accurate solution of the flow towards the building pit.

In [ ]:
n = 11  # number of layers around bottom of sheetpile wall

In [ ]:
dz_i_top = (z_well - z_dw) / np.sum(np.arange(n + 1))
dz_i_bot = (z_dw - z_extra) / np.sum(np.arange(2 * n + 1))

In [ ]:
z_layers_top = np.cumsum(np.arange(n) * dz_i_top)
z_layers_bot = np.cumsum(np.arange(2 * n) * dz_i_bot)

In [ ]:
zgr = np.r_[z_dw + z_layers_top[::-1], (z_dw - z_layers_bot)[1:]]

In [ ]:
z4 = np.r_[
    np.array([ztop + 1, ztop, z_well, z_well]),
    np.repeat(zgr, 2, 0),
    z_extra,
    z_extra,
    zbot,
]

In [ ]:
dz4 = z4[1:-1:2] - z4[2::2]

In [ ]:
kh_arr = kh * np.ones(dz4.shape)

In [ ]:
c = np.r_[np.array([ctop]), dz4[:-1] / (2 * kv) + dz4[1:] / (2 * kv)]

In [ ]:
kh_arr2 = kh_arr.copy()
kh_arr2[0] = 1e-5

Build model, solve, and calculate total discharge and distance to the 5 cm drawdown contour.

In [ ]:
ml = tml.ModelMaq(kaq=kh_arr, z=z4, c=c, topboundary="semi", hstar=0.0)

layers = np.arange(np.sum(z_dw <= ml.aq.zaqbot))
last_lay_dw = layers[-1]
inhom = tml.BuildingPitMaq(
    ml,
    xy,
    kaq=kh_arr2,
    z=z4[1:],
    topboundary="conf",
    c=c[1:],
    order=4,
    ndeg=3,
    layers=layers,
)

wlayers = np.arange(np.sum(-14 <= ml.aq.zaqbot))
wlayers = wlayers[1:]

tml.HeadLineSink(
    ml,
    x1=-length / 2 + offset,
    y1=width / 2 - offset,
    x2=length / 2 - offset,
    y2=width / 2 - offset,
    hls=h_bem,
    layers=wlayers,
)
tml.HeadLineSink(
    ml,
    x1=-length / 2 + offset,
    y1=0,
    x2=length / 2 - offset,
    y2=0,
    hls=h_bem,
    layers=wlayers,
    order=5,
)
tml.HeadLineSink(
    ml,
    x1=-length / 2 + offset,
    y1=-width / 2 + offset,
    x2=length / 2 - offset,
    y2=-width / 2 + offset,
    hls=h_bem,
    layers=wlayers,
)

# ml.solve_mp(nproc=2)
ml.solve()

Qtot = 0.0

for e in ml.elementlist:
    if e.name == "HeadLineSink":
        Qtot += e.discharge()

print("\nDischarge =", np.round(Qtot.sum(), 2), "m^3/dag")

y = np.linspace(-width / 2 - 25, width / 2 + 1100, 201)
hl = ml.headalongline(np.zeros(201), y, layers=[0])
y_5cm = np.interp(
    -0.05, ml.headalongline(np.zeros(201), y, layers=0).squeeze(), y, right=np.nan
)
print("Distance to 5 cm drawdown contour =", np.round(y_5cm, 2), "m")

In [ ]:
last_lay_dw = layers[-1]

In [ ]:
x = np.linspace(0.0, length / 2 + 1100, 201)
hl = ml.headalongline(x, np.zeros(201), layers=[0, last_lay_dw, last_lay_dw + 1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))

ax.plot(x, hl[0].squeeze(), label="head layer 0")
ax.plot(x, hl[1].squeeze(), label="head layer {}".format(last_lay_dw))
ax.plot(x, hl[2].squeeze(), label="head layer {}".format(last_lay_dw + 1))
ax.axhline(-0.05, color="r", linestyle="dashed", lw=0.75, label="-0.05 m")
ax.axhline(-0.5, color="k", linestyle="dashed", lw=0.75, label="-0.5 m")
ax.set_xlabel("x (m)")
ax.set_ylabel("head (m)")
ax.legend(loc="best")
ax.grid(True)

In [ ]:
x = np.linspace(-length / 2 - 25, 0.0, 201)
hl = ml.headalongline(x, np.zeros(201), layers=[0, last_lay_dw, last_lay_dw + 1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))

ax.plot(x, hl[0].squeeze(), label="head layer 0")
ax.plot(x, hl[1].squeeze(), label="head layer {}".format(last_lay_dw))
ax.plot(x, hl[2].squeeze(), label="head layer {}".format(last_lay_dw + 1))
ax.axhline(-0.05, color="r", linestyle="dashed", lw=0.75, label="-0.05 m")
ax.axhline(-0.5, color="k", linestyle="dashed", lw=0.75, label="-0.5 m")
ax.set_xlabel("x (m)")
ax.set_ylabel("head (m)")
ax.legend(loc="best")
ax.grid(True)

In [ ]:
xoffset = 50
zoffset = 15
x1, x2, y1, y2 = [-length / 2 - xoffset, 0.0, 0, 0]
nudge = 1e-6
n = 301

In [ ]:
# plot head contour cross-sections
h = ml.headalongline(
    np.linspace(x1 + nudge, x2 - nudge, n), np.linspace(y1 + nudge, y2 - nudge, n)
)
L = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
xg = np.linspace(0, L, n) + x1

zg = 0.5 * (ml.aq.zaqbot + ml.aq.zaqtop)
zg = np.hstack((ml.aq.zaqtop[0], zg, ml.aq.zaqbot[-1]))
h = np.vstack((h[0], h, h[-1]))

In [ ]:
levels = np.linspace(h_bem - 0.1, -0.0, 51)

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.set_aspect("equal")
ml.plots.topview(win=[x1, x2, y1, y2], orientation="ver", newfig=False)
cf = ax.contourf(xg, zg, h, levels)
cs = ax.contour(xg, zg, h, levels, colors="k", linewidths=0.5)
ax.set_ylim(z_dw - zoffset, z_dw + zoffset)
ax.set_ylabel("depth (m NAP)")
ax.set_xlabel("m")

cb = plt.colorbar(cf, ax=ax, shrink=0.6)
cb.ax.set_ylabel("head (m)")
cb.set_ticks(np.arange(-6, 0.1, 1))